In [1]:
import os
import torch
import pickle

from torchinfo import summary

In [2]:
from models import (
    make_slow_pretrained_body,
    make_slow_pretrained_classifier,
    make_slow_pretrained_segmenter,
    SegmentationHead)
from lared_laughter.constants import models_path

In [3]:
b = make_slow_pretrained_body()
s = SegmentationHead(output_len=45)

loaded pre-trained model
missing keys []
unexpected keys ['model.blocks.5.proj.weight', 'model.blocks.5.proj.bias']


In [4]:
input = [
    torch.rand((8, 3, 8, 256, 256)), # slow
    torch.rand((8, 3, 32, 256, 256)) # fast
]

In [6]:
# summary(pretrained, input_data=input, device='cuda', depth=5)
summary(b, input_size=(16, 3, 8, 256, 256), device='cuda', depth=5)

Layer (type:depth-idx)                                  Output Shape              Param #
PTVResNet                                               [16, 2048, 1, 2, 2]       --
├─AvgPool3d: 1-3                                        [16, 2048, 1, 2, 2]       --
├─Net: 1-2                                              [16, 2048, 8, 8, 8]       --
│    └─ModuleList: 2-1                                  --                        --
│    │    └─ResNetBasicStem: 3-1                        [16, 64, 8, 64, 64]       --
│    │    │    └─Conv3d: 4-1                            [16, 64, 8, 128, 128]     (9,408)
│    │    │    └─BatchNorm3d: 4-2                       [16, 64, 8, 128, 128]     (128)
│    │    │    └─ReLU: 4-3                              [16, 64, 8, 128, 128]     --
│    │    │    └─MaxPool3d: 4-4                         [16, 64, 8, 64, 64]       --
│    │    └─ResStage: 3-2                               [16, 256, 8, 64, 64]      --
│    │    │    └─ModuleList: 4                      

In [8]:
summary(s, input_size=(16, 2048, 8, 8, 8), device='cuda', depth=5)

/home/jose/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Layer (type:depth-idx)                   Output Shape              Param #
SegmentationHead                         [16, 45]                  --
├─Sequential: 1-1                        [16, 1, 8, 1, 1]          --
│    └─AvgPool3d: 2-1                    [16, 2048, 8, 1, 1]       --
│    └─Conv3d: 2-2                       [16, 1, 8, 1, 1]          6,145
├─Sequential: 1-2                        [16, 1, 45]               --
│    └─Flatten: 2-3                      [16, 1, 8]                --
│    └─Upsample: 2-4                     [16, 1, 45]               --
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
Total mult-adds (M): 0.79
Input size (MB): 67.11
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 67.13